# Project: Advanced Lane Finding - Main

### <font color='red'> Note: Run 'Advanced_LaneFinding - PreProcessing' to generate camera calibration, perspective transform matrices before running this! </font>

### Pipeline procedure
1. Load camera calibration, perspective transform matrices
2. Gradient & color filtering
3. Undistort & unwarp
4. Find lane lines:
    a) Use sliding window for first image or when reset is needed
    b) Use lookback for subsequent images

### Import libraries

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pickle

%matplotlib inline

In [3]:
try:
    with open('cameraCalibration_perspectiveTransform_matrices.pkl', 'rb') as f:
        data = pickle.load(f)
    mtx = data['mtx']
    dist = data['dist']
    M = data['M']
except FileNotFoundError:
    print("Run 'Advanced_LaneFinding - PreProcessing' to generate camera calibration, ",
          "perspective transform matrices before running this!")

def apply_gradx_color_filter(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    """Apply sobel filter along x and s-channel color filter"""
    img = np.copy(img)

    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # # Stack each channel
    # color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    combined = np.zeros_like(s_channel)
    combined[((sxbinary == 1)  | (s_binary == 1))] = 1

    return combined

def undistort_image(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

def warp_perspective(img, M):
    """
    `img` is the undistored image
    `M` is the perspective transform matrix
    """
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]), 
                                      flags=cv2.INTER_LINEAR)
    return warped

def find_lane_pixels(binary_warped, nwindows=9, margin=100, min_pixels=50):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height

        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this

        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
                              (win_xleft_high,win_y_high),
                      (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
                              (win_xright_high,win_y_high),
                      (0,255,0), 2)

        ### indIdentify the nonzero pixels in x and y within the window ###
        good_left_inds = np.where((nonzerox >= win_xleft_low) &
                                  (nonzerox < win_xleft_high) &
                                  (nonzeroy >= win_y_low) &
                                  (nonzeroy < win_y_high))[0]
        # list(zip(nonzerox[nz_left], nonzeroy[nz_left]))
        good_right_inds = np.where((nonzerox >= win_xright_low) &
                                  (nonzerox < win_xright_high) &
                                  (nonzeroy >= win_y_low) &
                                  (nonzeroy < win_y_high))[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        ### If you found > min_pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > min_pixels:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))

        if len(good_right_inds) > min_pixels:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))


    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def search_around_poly(binary_warped, left_fit, right_fit, margin=100):
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = np.where((nonzerox >= (np.polyval(left_fit, nonzeroy) - margin)) &
                              (nonzerox <= (np.polyval(left_fit, nonzeroy) + margin)))
    right_lane_inds = np.where((nonzerox >= (np.polyval(right_fit, nonzeroy) - margin)) &
                              (nonzerox <= (np.polyval(right_fit, nonzeroy) + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

def fit_lane_polynomials(ylen, leftx, lefty, rightx, righty, order=2):
    ### Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, order)
    right_fit = np.polyfit(righty, rightx, order)

    # Generate x and y values for plotting
    ploty = np.linspace(0, ylen-1, ylen)
    left_fitx = np.polyval(left_fit, ploty)
    right_fitx = np.polyval(right_fit, ploty)

    return left_fit, right_fit, left_fitx, right_fitx, ploty

def get_poly_lanes(img, prev_left_fit=None, prev_right_fit=None):
    # Apply Sobel Gradient filter, color filter
    img_filt = apply_gradx_color_filter(img)

    # Undistort and unwarp image
    img_undistort = undistort_image(img=img_filt, mtx=mtx, dist=dist)
    img_unwarp = warp_perspective(img=img_undistort, M=M)

    # Find lane pixels
    if prev_left_fit is None or prev_right_fit is None:
        leftx, lefty, rightx, righty, out_img = find_lane_pixels(img_unwarp)
    else:
        # detect lines around previous fit
        leftx, lefty, rightx, righty = search_around_poly(img_unwarp, left_fit=prev_left_fit,
                                                          right_fit=prev_right_fit, margin=50)

    # Fit polynomials lines to left, right lanes
    left_fit, right_fit, left_fitx, right_fitx, ploty = fit_lane_polynomials(img.shape[0], leftx, lefty, rightx, righty)

    return img_unwarp, left_fit, right_fit, left_fitx, right_fitx, ploty

def plot_lane_lines(img, left_x, right_x, y):
    for i in range(len(left_x)-1):
        cv2.line(img, (int(left_x[i]), int(y[i+1])), (int(left_x[i+1]), int(y[i+1])), color=(255, 0, 0), thickness=5)
        cv2.line(img, (int(right_x[i]), int(y[i+1])), (int(right_x[i+1]), int(y[i+1])), color=(255, 0, 0), thickness=5)

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.

    `initial_img` should be the image before any processing.

    The result image is computed as follows:

    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

def add_lanes_poly(image, left_fitx, right_fitx, y):
    # create lanes poly
    pts = np.vstack([np.vstack([left_fitx, y]).astype(np.int32).T, 
                     np.vstack([right_fitx[::-1], y[::-1]]).astype(np.int32).T])
    lanes_poly = np.zeros([image.shape[0], image.shape[1], 3])
    lanes_img = cv2.fillPoly(lanes_poly,  [pts],  (255, 0, 0))

    # unwarp lanes perspective
    lanes_img_warp = warp_perspective(img=lanes_img, M=np.linalg.inv(M)).astype(np.uint8)


    return weighted_img(img=lanes_img_warp, initial_img=image, α=1.0, β=0.5)

def calc_poly_curve_radius(poly_fit, y_eval):
    return ((1 + (2*poly_fit[0]*y_eval+poly_fit[1])**2)**1.5)/(2*abs(poly_fit[0]))

def calc_lane_curve_radius(poly_fit, y_eval, xm_per_pix, ym_per_pix):
    # convert poly fit from pixels to m
    real_fit = np.copy(poly_fit)
    real_fit[0] *= xm_per_pix/(ym_per_pix**2)
    real_fit[1] *= (xm_per_pix/ym_per_pix)
    y_eval_m = y_eval*ym_per_pix
    
    return calc_poly_curve_radius(real_fit, y_eval_m)

class LaneFinding():
    def __init__(self):
        self.ym_per_pix = 30/720.
        self.xm_per_pix = 3.7/700.
        self.n_hist = 15
        
        # was the line detected in the last iteration?
        self.detected = False  
        
        self.best_x_vals = [None, None]
        self.recent_x_vals = [[None, None]]
        
        self.best_fits = [None, None]  
        self.recent_fits = [[None, None]]
        
        #radius of curvature of the line in some units
        self.radius_of_curvatures = [[None, None]]
        self.best_radius = None
        
        #distance in meters of vehicle center from the line
        self.line_base_pos = [None]
        
    def update_best_fits(self):
        if len(self.recent_fits) > self.n_hist:
            self.recent_fits = self.recent_fits[-self.n_hist:]
            self.recent_x_vals = self.recent_x_vals[-self.n_hist:]
            self.radius_of_curvatures = self.radius_of_curvatures[-self.n_hist:]
            self.line_base_pos = self.line_base_pos[-self.n_hist:]
        
        best_fit_left = np.mean(np.array([fits[0] for fits in self.recent_fits if fits[0] is not None]), axis=0)
        best_fit_right = np.mean(np.array([fits[1] for fits in self.recent_fits if fits[1] is not None]), axis=0)
        if not np.all(np.isnan(best_fit_left)) and not np.all(np.isnan(best_fit_right)):
            self.best_fits = [best_fit_left, best_fit_right]
        else:
            self.best_fits = [None, None]
        
        best_xvals_left = np.mean(np.array([xvals[0] for xvals in self.recent_x_vals if xvals[0] is not None]), axis=0)
        best_xvals_right = np.mean(np.array([xvals[1] for xvals in self.recent_x_vals if xvals[1] is not None]), axis=0)
        if not np.all(np.isnan(best_xvals_left)) and not np.all(np.isnan(best_xvals_left)):
            self.best_x_vals = [best_xvals_left, best_xvals_right]
        else:
            self.best_x_vals = [None, None]
        
        avg_radius_left = np.mean(np.array([radii[0] for radii in self.radius_of_curvatures if radii[0] is not None]), axis=0)
        avg_radius_right = np.mean(np.array([radii[1] for radii in self.radius_of_curvatures if radii[1] is not None]), axis=0)
        self.best_radius = 0.5*(avg_radius_left + avg_radius_right)
        
    
    def process_image(self, image):
        img_unwarp, left_fit, right_fit, left_fitx, right_fitx, ploty = get_poly_lanes(image, prev_left_fit=self.best_fits[0],
                                                                                       prev_right_fit=self.best_fits[1])
        
        left_curv_radius = calc_lane_curve_radius(left_fit, image.shape[0], 
                                                  self.xm_per_pix, self.ym_per_pix)
        right_curv_radius = calc_lane_curve_radius(right_fit, image.shape[0], 
                                                   self.xm_per_pix, self.ym_per_pix)
        
        curve_ratio = left_curv_radius/right_curv_radius
        lane_diff = (right_fitx - left_fitx) * self.xm_per_pix
        
        lane_width_avg = np.mean(lane_diff)
        lane_width_max = np.max(lane_diff)
        lane_width_min = np.min(lane_diff)
        
        if min(left_curv_radius, right_curv_radius) > 1000:
            curve_ratio = 1.
            
        if (curve_ratio > 0.5) and (curve_ratio < 2) and (lane_width_avg > 3.55) and \
            (lane_width_avg < 3.85) and ((lane_width_max - lane_width_min) < 0.5):
            # fit looks good!
            self.recent_fits.append([left_fit, right_fit])
            self.recent_x_vals.append([left_fitx, right_fitx])
            self.radius_of_curvatures.append([left_curv_radius, right_curv_radius])
        else:
            # fit looks bad!
            self.recent_fits.append([None, None])
            self.recent_x_vals.append([None, None])
            self.radius_of_curvatures.append([None, None])
            
            print("Lane width mean = ", lane_width_avg)
            print("Lane width min = ", lane_width_min)
            print("Lane width max = ", lane_width_max)
            print("Curve Ratio = ", curve_ratio)
        
        self.update_best_fits()

        if self.best_x_vals[0] is not None:
            img_wlanes = add_lanes_poly(image=image, left_fitx=self.best_x_vals[0], right_fitx=self.best_x_vals[1], y=ploty)
        else:
            img_wlanes = add_lanes_poly(image=image, left_fitx=left_fitx, right_fitx=right_fitx, y=ploty)

        font                   = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (50, 50)
        bottomLeftCornerOfText2 = (50, 100)
        bottomLeftCornerOfText3 = (50, 150)
        fontScale              = 1
        fontColor              = (255,255,255)
        lineType               = 2
        
        cv2.putText(img_wlanes, "Radius of Curvature (Left) = {0:.1f}".format(left_curv_radius),
                    bottomLeftCornerOfText, font, fontScale, fontColor, lineType)        
        cv2.putText(img_wlanes, "Radius of Curvature (Right) = {0:.1f}".format(right_curv_radius),
                    bottomLeftCornerOfText2, font, fontScale, fontColor, lineType)
        cv2.putText(img_wlanes, "Radius of Curvature = {0:.1f}".format(self.best_radius),
                    bottomLeftCornerOfText3, font, fontScale, fontColor, lineType)
        return img_wlanes

### Testing Video

In [4]:
lane_detection = LaneFinding()

In [5]:
from moviepy.editor import VideoFileClip
white_output = 'project_video_out.mp4'
clip1 = VideoFileClip("project_video.mp4")#.subclip(18,25)
white_clip = clip1.fl_image(lane_detection.process_image)
%time white_clip.write_videofile(white_output, audio=False)
clip1.close()

Moviepy - Building video project_video_out.mp4.
Moviepy - Writing video project_video_out.mp4



t:   2%|█▍                                                                 | 28/1260 [00:03<02:49,  7.26it/s, now=None]

Lane width mean =  3.87215012994
Lane width min =  3.72434011093
Lane width max =  3.9913397332
Curve Ratio =  1.08014663492


t:   5%|███▏                                                               | 61/1260 [00:09<03:14,  6.15it/s, now=None]

Lane width mean =  3.54814669332
Lane width min =  3.49198772375
Lane width max =  3.60470959536
Curve Ratio =  0.998584718439


...

t: 100%|████████████████████████████████████████████████████████████████▉| 1259/1260 [03:06<00:00,  6.91it/s, now=None]

Lane width mean =  3.89850654063
Lane width min =  3.72427619594
Lane width max =  4.1244605633
Curve Ratio =  1.0


t: 100%|█████████████████████████████████████████████████████████████████| 1260/1260 [03:06<00:00,  6.65it/s, now=None]

Lane width mean =  3.90010322085
Lane width min =  3.73170188912
Lane width max =  4.13199616651
Curve Ratio =  1.0


Moviepy - Done !
Moviepy - video ready project_video_out.mp4
Wall time: 3min 8s


In [ ]:
from moviepy.editor import VideoFileClip
clip1 = VideoFileClip("project_video.mp4").subclip(18,25)

In [ ]:
plt.imshow(lane_detection.process_image(clip1.get_frame(2/25.)))